In [ ]:
import torch
from torch.utils.data import DataLoader as dl
from torch import nn

from torchvision import datasets as dt
from torchvision.transforms import ToTensor,Lambda,Compose
import matplotlib.pyplot as plt

tr = dt.FashionMNIST(root='fa',download=True,train=True ,transform =ToTensor() )
te = dt.FashionMNIST(root='fa',download=True,train=False,transform =ToTensor() )

trdl = dl(tr,128)
tedl = dl(te,128)

class ypnet(nn.Module):
    def __init__(self):
        super(ypnet,self).__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784,392),
            nn.ReLU(),
            nn.Linear(392,156),
            nn.ReLU(),
            nn.Linear(156,78),
            nn.ReLU(),
            nn.Linear(78,10),
            nn.Softmax(dim=1)
        )
    def forward(self,x):
        return self.net(x)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
yp = ypnet().to(device)
loss = nn.CrossEntropyLoss()
opt = torch.optim.Adam(yp.parameters() )

def train(dl,model,lossf,opt):
    for x,y in dl:
        x,y = x.to(device),y.to(device)
        pre = model(x)
        loss = lossf(pre,y)

        opt.zero_grad()
        loss.backward()
        opt.step()

def test(dl,model,lossf):
    model.eval() # eval scope
    size,losses,corrects = len(dl.dataset),0,0

    with torch.no_grad():
        for x,y in dl:
            x,y = x.to(device),y.to(device)
            pre = model(x)
            loss = lossf(pre,y)

            losses += loss.item()
            corrects += (pre.argmax(1)==y).type(torch.float).sum().item()
    print(f'accuracy {corrects/size} loss {losses/size}')
for _ in range(10):
    train(trdl,yp,loss,opt)
    test(tedl,yp,loss)